In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,OrdinalEncoder,FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler,RobustScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression

import seaborn as sns


In [112]:
# load the data
df = pd.read_csv("Student_dataset_3.csv")

# rename the columns
df.columns=['RollNo','Gender','Age','Location','Famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','traveltime','studytime','Failures','Famsup','Paid','Activities','HigherEdu','Internet','Famrel','Freetime','GoOut','Health','10th%','12thordiploma%','EAMCETorECETrank','Internal','Prev cgpa','sgpa']

# remove duplicates 
df = df.drop_duplicates()




In [113]:
df['Eamcet Rank'] = 0
df['Ecet Rank'] = 0

df.loc[~df['RollNo'].str.contains('L'), 'Eamcet Rank'] = df.loc[~df['RollNo'].str.contains('L'), 'EAMCETorECETrank']
df.loc[df['RollNo'].str.contains('L'), 'Ecet Rank'] = df.loc[df['RollNo'].str.contains('L'), 'EAMCETorECETrank']
df.drop('EAMCETorECETrank', axis=1, inplace=True)


In [114]:
df['Prev cgpa'] = df['Prev cgpa'].replace(0, 5.5)
df['sgpa'] = df['sgpa'].replace(0, 5.5)
df['Total_eamcet/ecet_grade'] = 0


In [115]:
df['Fedu'] = df['Fedu'].replace('primary education (4th grade)', 'primary education( 4th grade )')

In [116]:
df['traveltime'] = df['traveltime'].replace('<1hr', '>1hr')

In [117]:
df.head(1)

,RollNo,Gender,Age,Location,Famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,traveltime,studytime,Failures,Famsup,Paid,Activities,HigherEdu,Internet,Famrel,Freetime,GoOut,Health,10th%,12thordiploma%,Internal,Prev cgpa,sgpa,Eamcet Rank,Ecet Rank,Total_eamcet/ecet_grade
0,319126510001,FEMALE,21,Urban,5,T,higher education,higher education,Homemaker,civil services: administrative or police,Other,30min - 1hr,1- <2 hr,0,Yes,No,Yes,No,Yes,excellent,high,3 - medium,5 - very good,93.1,96.3,90.23,9.4,8.93,18238.0,0,0


In [118]:
df.to_csv("cleaned_dataset.csv")

In [119]:

# assume the DataFrame is stored in a variable called "df"
unique_values = {}
for column in df.columns:
    values = df[column].unique()
    if len(values) < 10:
        unique_values[column] = values

# print the unique values for each column
for column, values in unique_values.items():
    print(f"Column '{column}': {values}")
    print()


Column 'Gender': ['FEMALE' 'MALE']

Column 'Age': [21 22 20]

Column 'Location': ['Urban' 'Rural']

Column 'Famsize': [5 4 3 2 6 8 9]

Column 'Pstatus': ['T' 'A']

Column 'Medu': ['higher education' '5th to 9th grade' 'secondary education' 'None'
 'primary education( 4th grade )']

Column 'Fedu': ['higher education' 'secondary education' '5th to 9th grade'
 'primary education( 4th grade )' 'None']

Column 'Mjob': ['Homemaker' "nominal: 'teacher', 'health' care related"
 'civil services: administrative or police']

Column 'Fjob': ['civil services: administrative or police'
 "nominal: 'teacher', 'health' care related" 'Homemaker']

Column 'reason': ['Other' 'Course preference' "Institution's reputation" 'close to home']

Column 'traveltime': ['30min - 1hr' '>1hr' '15 - 30 min' '<15 min']

Column 'studytime': ['1- <2 hr' '2 to 5 hr' '>10 hr' '5 to 10 hr']

Column 'Failures': [0 2 3 9 5 4 1]

Column 'Famsup': ['Yes' 'No']

Column 'Paid': ['No' 'Yes']

Column 'Activities': ['Yes' 'No']

Col

In [120]:
X = df.drop('sgpa', axis=1)
y = df['sgpa']

X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['sgpa']),df['sgpa'],test_size=0.2,random_state=42)


In [121]:
X.columns

Index(['RollNo', 'Gender', 'Age', 'Location', 'Famsize', 'Pstatus', 'Medu',
       'Fedu', 'Mjob', 'Fjob', 'reason', 'traveltime', 'studytime', 'Failures',
       'Famsup', 'Paid', 'Activities', 'HigherEdu', 'Internet', 'Famrel',
       'Freetime', 'GoOut', 'Health', '10th%', '12thordiploma%', 'Internal',
       'Prev cgpa', 'Eamcet Rank', 'Ecet Rank', 'Total_eamcet/ecet_grade'],
      dtype='object')

In [122]:
X.iloc[:1]

,RollNo,Gender,Age,Location,Famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,traveltime,studytime,Failures,Famsup,Paid,Activities,HigherEdu,Internet,Famrel,Freetime,GoOut,Health,10th%,12thordiploma%,Internal,Prev cgpa,Eamcet Rank,Ecet Rank,Total_eamcet/ecet_grade
0,319126510001,FEMALE,21,Urban,5,T,higher education,higher education,Homemaker,civil services: administrative or police,Other,30min - 1hr,1- <2 hr,0,Yes,No,Yes,No,Yes,excellent,high,3 - medium,5 - very good,93.1,96.3,90.23,9.4,18238.0,0,0


In [123]:
def get_eamcet_grade(rank):
    conditions = [
        rank == 0,
        rank < 7000,
        rank < 8500,
        rank < 10000,
        rank < 25000
    ]
    choices = [0, 4, 3, 2, 1]
    return np.select(conditions, choices, default=0)

def get_ecet_grade(rank):
    conditions = [
        rank == 0,
        rank < 250,
        rank < 350,
        rank < 580,
        rank < 1000
    ]
    choices = [0, 4, 3, 2, 1]
    return np.select(conditions, choices, default=0)

ctf1 = ColumnTransformer([
        ('imputer1', SimpleImputer(strategy='mean'), [28]),  #Eamcet Rank
         ('Eamcet_transformer', FunctionTransformer(get_eamcet_grade), [28]),
        ('imputer2', SimpleImputer(strategy='mean'), [29]),  #Ecet Rank
        ('Ecet_transformer', FunctionTransformer(get_ecet_grade), [29]),
    ],
    remainder='passthrough'
)


df_transformed = ctf1.fit_transform(df)
print(df_transformed[0])


column_names = [ 'Eamcet Rank','Eamcet Grade', 'Ecet Rank','Ecet Grade','Roll No','Gender', 'Age', 
                'Location', 'Famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'traveltime',
                'studytime', 'Failures', 'Famsup', 'Paid', 'Activities', 'HigherEdu', 'Internet', 'Famrel',
                'Freetime', 'GoOut', 'Health', '10th%', '12thordiploma%', 'Internal', 'Prev cgpa','sgpa','Total_eamcet/ecet_grade']

df = pd.DataFrame(df_transformed, columns=column_names)

df.head(1)



[18238.0 1 0.0 0 '319126510001' 'FEMALE' 21 'Urban' 5 'T'
 'higher education' 'higher education' 'Homemaker'
 'civil services: administrative or police' 'Other' '30min - 1hr'
 '1- <2 hr' 0 'Yes' 'No' 'Yes' 'No' 'Yes' 'excellent' 'high' '3 - medium'
 '5 - very good' 93.1 96.3 90.23 9.4 8.93 0]


,Eamcet Rank,Eamcet Grade,Ecet Rank,Ecet Grade,Roll No,Gender,Age,Location,Famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,traveltime,studytime,Failures,Famsup,Paid,Activities,HigherEdu,Internet,Famrel,Freetime,GoOut,Health,10th%,12thordiploma%,Internal,Prev cgpa,sgpa,Total_eamcet/ecet_grade
0,18238.0,1,0.0,0,319126510001,FEMALE,21,Urban,5,T,higher education,higher education,Homemaker,civil services: administrative or police,Other,30min - 1hr,1- <2 hr,0,Yes,No,Yes,No,Yes,excellent,high,3 - medium,5 - very good,93.1,96.3,90.23,9.4,8.93,0


In [124]:
index = df.columns.get_loc('Total_eamcet/ecet_grade')
print(index)


32


In [125]:
print(df.columns)


Index(['Eamcet Rank', 'Eamcet Grade', 'Ecet Rank', 'Ecet Grade', 'Roll No',
       'Gender', 'Age', 'Location', 'Famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'traveltime', 'studytime', 'Failures',
       'Famsup', 'Paid', 'Activities', 'HigherEdu', 'Internet', 'Famrel',
       'Freetime', 'GoOut', 'Health', '10th%', '12thordiploma%', 'Internal',
       'Prev cgpa', 'sgpa', 'Total_eamcet/ecet_grade'],
      dtype='object')


In [126]:
def second_transformer(X):
    # Add values at indices 1 and 3
    X[:, 1] = X[:, 1] + X[:, 3]

    # Replace the value at the last index with the sum of values at indices 1 and 3
    X[:, -1] = X[:, 1] + X[:, 3]

    return X

ctf2 = FunctionTransformer(second_transformer)

df_trans2 = ctf2.fit_transform(df_transformed)
df1 = pd.DataFrame(df_trans2, columns=['Eamcet Rank', 'Eamcet Grade', 'Ecet Rank', 'Ecet Grade', 'Roll No',
       'Gender', 'Age', 'Location', 'Famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'traveltime', 'studytime', 'Failures',
       'Famsup', 'Paid', 'Activities', 'HigherEdu', 'Internet', 'Famrel',
       'Freetime', 'GoOut', 'Health', '10th%', '12thordiploma%', 'Internal',
       'Prev cgpa', 'sgpa', 'Total_eamcet/ecet_grade'])

df1.head(1)


,Eamcet Rank,Eamcet Grade,Ecet Rank,Ecet Grade,Roll No,Gender,Age,Location,Famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,traveltime,studytime,Failures,Famsup,Paid,Activities,HigherEdu,Internet,Famrel,Freetime,GoOut,Health,10th%,12thordiploma%,Internal,Prev cgpa,sgpa,Total_eamcet/ecet_grade
0,18238.0,1,0.0,0,319126510001,FEMALE,21,Urban,5,T,higher education,higher education,Homemaker,civil services: administrative or police,Other,30min - 1hr,1- <2 hr,0,Yes,No,Yes,No,Yes,excellent,high,3 - medium,5 - very good,93.1,96.3,90.23,9.4,8.93,1


In [127]:
# df1

In [128]:
# X_test

In [129]:
categories_medu = ['None', 'primary education( 4th grade )', '5th to 9th grade', 'secondary education', 'higher education']
categories_fedu = ['None', 'primary education( 4th grade )', '5th to 9th grade', 'secondary education', 'higher education']
categories_travel=[ '<15 min','15 - 30 min','30min - 1hr','>1hr']
categories_study=[ '1- <2 hr', '2 to 5 hr','5 to 10 hr', '>10 hr']
categories_famrel=[ 'Bad','Good','Very good','excellent']
categories_freetime=['very low', 'low', 'high','very high']
categories_goout=[ '1 - very low', '2 - low', '3 - medium','4 - high','5 - very high']
categories_health=[ '1 - very bad', '2- bad', '3 - average','4 - good','5 - very good']

# Define the column transformer with ordinal encoding for Medu and Fedu
ctf3 = ColumnTransformer(transformers=[('ordinal', OrdinalEncoder(categories=[categories_medu,
                            categories_fedu,categories_travel,categories_study,categories_famrel,
                            categories_freetime,categories_goout,categories_health],dtype=np.int32),
                                        [10,11,15,16,23,24,25,26])], remainder='passthrough')



df1_transformed = ctf3.fit_transform(df1)

df1_transformed[0]
df2 = pd.DataFrame(df1_transformed, columns=['Medu', 'Fedu', 'traveltime', 'studytime','Famrel','Freetime',
                                             'GoOut','Health','Eamcet Rank','Eamcet Grade','Ecet Rank','Ecet Grade','RollNo', 'Gender', 'Age', 'Location', 'Famsize',
                                             'Pstatus', 'Mjob', 'Fjob', 'reason', 'Failures', 'Famsup', 'Paid', 
                                             'Activities', 'HigherEdu', 'Internet','10th%', '12thordiploma%', 
                                             'Internal','Prev cgpa', 'sgpa','Total_eamcet/ecet_grade'])



df2

,Medu,Fedu,traveltime,studytime,Famrel,Freetime,GoOut,Health,Eamcet Rank,Eamcet Grade,Ecet Rank,Ecet Grade,RollNo,Gender,Age,Location,Famsize,Pstatus,Mjob,Fjob,reason,Failures,Famsup,Paid,Activities,HigherEdu,Internet,10th%,12thordiploma%,Internal,Prev cgpa,sgpa,Total_eamcet/ecet_grade
0,4,4,2,0,3,2,2,4,18238.0,1,0.0,0,319126510001,FEMALE,21,Urban,5,T,Homemaker,civil services: administrative or police,Other,0,Yes,No,Yes,No,Yes,93.1,96.3,90.23,9.4,8.93,1
1,4,4,3,1,3,2,1,3,7586.0,3,0.0,0,319126510003,FEMALE,22,Urban,4,T,"nominal: 'teacher', 'health' care related","nominal: 'teacher', 'health' care related",Course preference,0,Yes,Yes,Yes,No,No,86.0,95.6,89.77,9.07,8.8,3
2,2,3,3,3,3,0,0,4,8791.0,2,0.0,0,319126510004,MALE,21,Urban,4,T,Homemaker,"nominal: 'teacher', 'health' care related",Institution's reputation,0,Yes,No,No,No,Yes,90.0,93.0,71.14,8.07,6.73,2
3,3,4,3,1,3,2,3,3,7844.0,3,0.0,0,319126510005,MALE,21,Urban,4,T,Homemaker,"nominal: 'teacher', 'health' care related",Institution's reputation,0,Yes,No,Yes,No,Yes,92.15,95.0,81.14,9.13,8.53,3
4,3,3,2,1,3,1,1,3,103750.0,0,0.0,0,319126510006,FEMALE,20,Urban,4,T,Homemaker,civil services: administrative or police,Course preference,2,Yes,No,Yes,Yes,Yes,90.25,76.5,77.05,8.0,7.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2,3,3,1,2,2,3,3,0.0,3,323.0,3,320126510L06,FEMALE,21,Urban,4,T,Homemaker,Homemaker,Institution's reputation,0,Yes,No,No,Yes,Yes,88.35,78.0,84.77,8.87,8.33,6
95,3,3,3,2,1,1,1,3,0.0,1,951.0,1,320126510L10,FEMALE,21,Rural,4,A,Homemaker,Homemaker,Course preference,0,Yes,Yes,No,No,No,80.75,80.0,81.82,7.4,7.53,2
96,3,3,2,1,2,2,2,2,0.0,4,239.0,4,320126510L11,FEMALE,21,Urban,4,T,Homemaker,civil services: administrative or police,Other,0,Yes,No,No,Yes,No,89.3,87.0,86.59,8.8,8.67,8
97,4,4,3,1,0,2,0,3,0.0,3,269.0,3,320126510L12,MALE,21,Rural,3,A,"nominal: 'teacher', 'health' care related","nominal: 'teacher', 'health' care related",Other,0,Yes,No,Yes,No,No,88.35,81.0,83.86,8.13,8.0,6


In [130]:
# define column transformer
ctf4 = ColumnTransformer(
    transformers=[
        ('one_hot', OneHotEncoder(drop='first', sparse=False, dtype=np.int32, handle_unknown='ignore'), [13,15,17,18, 19,20, 22, 23, 24, 25,26])
    ],
    remainder='passthrough')


# apply column transformer
df2_transformed = ctf4.fit_transform(df2)

df2_transformed.shape

# create dataframe with transformed data
df3 = pd.DataFrame(df2_transformed, columns=['Gender','Location_urban','Pstatus','Mjob_civil','Mjob_Teacher/health','Fjob_civil','Fjob_Teacher/health',
                                             'reason_reputation','reason_other','reason_closetohome','Famsup','Paid', 'Activities',
                                             'HigherEdu', 'Internet', 'Medu', 'Fedu', 'traveltime', 'studytime','Famrel','Freetime',
                                             'GoOut','Health','Eamcet Rank','Eamcet Grade','Ecet Rank','Ecet Grade','RollNo', 'Age',  "Famsize",
                                            'Failures','10th%', '12thordiploma%',
                                             'Internal','Prev cgpa', 'sgpa','Total_eamcet/ecet_grade'])

df3


,Gender,Location_urban,Pstatus,Mjob_civil,Mjob_Teacher/health,Fjob_civil,Fjob_Teacher/health,reason_reputation,reason_other,reason_closetohome,Famsup,Paid,Activities,HigherEdu,Internet,Medu,Fedu,traveltime,studytime,Famrel,Freetime,GoOut,Health,Eamcet Rank,Eamcet Grade,Ecet Rank,Ecet Grade,RollNo,Age,Famsize,Failures,10th%,12thordiploma%,Internal,Prev cgpa,sgpa,Total_eamcet/ecet_grade
0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,1,4,4,2,0,3,2,2,4,18238.0,1,0.0,0,319126510001,21,5,0,93.1,96.3,90.23,9.4,8.93,1
1,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,4,4,3,1,3,2,1,3,7586.0,3,0.0,0,319126510003,22,4,0,86.0,95.6,89.77,9.07,8.8,3
2,1,1,1,0,0,0,1,1,0,0,1,0,0,0,1,2,3,3,3,3,0,0,4,8791.0,2,0.0,0,319126510004,21,4,0,90.0,93.0,71.14,8.07,6.73,2
3,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,3,4,3,1,3,2,3,3,7844.0,3,0.0,0,319126510005,21,4,0,92.15,95.0,81.14,9.13,8.53,3
4,0,1,1,0,0,1,0,0,0,0,1,0,1,1,1,3,3,2,1,3,1,1,3,103750.0,0,0.0,0,319126510006,20,4,2,90.25,76.5,77.05,8.0,7.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,2,3,3,1,2,2,3,3,0.0,3,323.0,3,320126510L06,21,4,0,88.35,78.0,84.77,8.87,8.33,6
95,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3,3,3,2,1,1,1,3,0.0,1,951.0,1,320126510L10,21,4,0,80.75,80.0,81.82,7.4,7.53,2
96,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,3,3,2,1,2,2,2,2,0.0,4,239.0,4,320126510L11,21,4,0,89.3,87.0,86.59,8.8,8.67,8
97,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,4,4,3,1,0,2,0,3,0.0,3,269.0,3,320126510L12,21,3,0,88.35,81.0,83.86,8.13,8.0,6


In [131]:
# define the column transformer to apply scaling and capping
ctf5 = ColumnTransformer(transformers=[
    ('scaling', RobustScaler(), [31,32,33,34]),
    ('capping', QuantileTransformer(output_distribution='uniform', n_quantiles=60, random_state=42),[31,32,33,34])
], remainder='passthrough')

# fit and transform the data using the preprocessor
df3_transformed = ctf5.fit_transform(df3)
df3_transformed.shape


# print the shape of the transformed dataframe
df4 =pd.DataFrame(df3_transformed,columns=['scaled_10th%', 'scaled_12thordiploma%',
                                             'scaled_Internal','scaled_Prev cgpa','10th%', '12thordiploma%',
                                             'Internal','Prev cgpa','Gender','Location_urban','Pstatus','Mjob_civil','Mjob_Teacher/health','Fjob_civil','Fjob_Teacher/health',
                                             'reason_reputation','reason_other','reason_closetohome','Famsup','Paid', 'Activities',
                                             'HigherEdu', 'Internet', 'Medu', 'Fedu', 'traveltime', 'studytime','Famrel','Freetime',
                                             'GoOut','Health','Eamcet Rank','Eamcet Grade','Ecet Rank','Ecet Grade','RollNo', 'Age',  "Famsize",
                                            'Failures', 'sgpa','Total_eamcet/ecet_grade'])



df4.head()

,scaled_10th%,scaled_12thordiploma%,scaled_Internal,scaled_Prev cgpa,10th%,12thordiploma%,Internal,Prev cgpa,Gender,Location_urban,Pstatus,Mjob_civil,Mjob_Teacher/health,Fjob_civil,Fjob_Teacher/health,reason_reputation,reason_other,reason_closetohome,Famsup,Paid,Activities,HigherEdu,Internet,Medu,Fedu,traveltime,studytime,Famrel,Freetime,GoOut,Health,Eamcet Rank,Eamcet Grade,Ecet Rank,Ecet Grade,RollNo,Age,Famsize,Failures,sgpa,Total_eamcet/ecet_grade
0,0.0,0.163462,0.430099,0.484018,0.576271,0.745297,0.830508,0.864407,0,1,1,0,0,1,0,0,1,0,1,0,1,0,1,4,4,2,0,3,2,2,4,18238.0,1,0.0,0,319126510001,21,5,0,8.93,1
1,-1.067669,0.096154,0.39227,0.182648,0.183116,0.669492,0.762712,0.677966,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,4,4,3,1,3,2,1,3,7586.0,3,0.0,0,319126510003,22,4,0,8.8,3
2,-0.466165,-0.153846,-1.139803,-0.730594,0.300066,0.440678,0.118644,0.245763,1,1,1,0,0,0,1,1,0,0,1,0,0,0,1,2,3,3,3,3,0,0,4,8791.0,2,0.0,0,319126510004,21,4,0,6.73,2
3,-0.142857,0.038462,-0.317434,0.237443,0.415254,0.567797,0.334373,0.720339,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,3,4,3,1,3,2,3,3,7844.0,3,0.0,0,319126510005,21,4,0,8.53,3
4,-0.428571,-1.740385,-0.653783,-0.794521,0.338983,0.082452,0.227964,0.220339,0,1,1,0,0,1,0,0,0,0,1,0,1,1,1,3,3,2,1,3,1,1,3,103750.0,0,0.0,0,319126510006,20,4,2,7.13,0


In [132]:
# c=0
# for i in df4.columns:
#     print(i,c)
#     c+=1

In [133]:
cols_to_drop=[0,1,2,3,31,32,33,34,35]

# define the transformer to drop the specified columns
ctf6 = ColumnTransformer(transformers=[
    ('drop', 'drop', cols_to_drop)
], remainder='passthrough')

# fit and transform the data using the transformer
df4_transformed = ctf6.fit_transform(df4)

df5 =pd.DataFrame(df4_transformed,columns=['10th%', '12thordiploma%',
                                             'Internal','Prev cgpa','Gender','Location_urban','Pstatus','Mjob_civil','Mjob_Teacher/health','Fjob_civil','Fjob_Teacher/health',
                                             'reason_reputation','reason_other','reason_closetohome','Famsup','Paid', 'Activities',
                                             'HigherEdu', 'Internet', 'Medu', 'Fedu', 'traveltime', 'studytime','Famrel','Freetime',
                                             'GoOut','Health','Age',  "Famsize",
                                            'Failures', 'sgpa','Total_eamcet/ecet_grade'])

df5.head(5)

,10th%,12thordiploma%,Internal,Prev cgpa,Gender,Location_urban,Pstatus,Mjob_civil,Mjob_Teacher/health,Fjob_civil,Fjob_Teacher/health,reason_reputation,reason_other,reason_closetohome,Famsup,Paid,Activities,HigherEdu,Internet,Medu,Fedu,traveltime,studytime,Famrel,Freetime,GoOut,Health,Age,Famsize,Failures,sgpa,Total_eamcet/ecet_grade
0,0.576271,0.745297,0.830508,0.864407,0,1,1,0,0,1,0,0,1,0,1,0,1,0,1,4,4,2,0,3,2,2,4,21,5,0,8.93,1
1,0.183116,0.669492,0.762712,0.677966,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,4,4,3,1,3,2,1,3,22,4,0,8.8,3
2,0.300066,0.440678,0.118644,0.245763,1,1,1,0,0,0,1,1,0,0,1,0,0,0,1,2,3,3,3,3,0,0,4,21,4,0,6.73,2
3,0.415254,0.567797,0.334373,0.720339,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,3,4,3,1,3,2,3,3,21,4,0,8.53,3
4,0.338983,0.082452,0.227964,0.220339,0,1,1,0,0,1,0,0,0,0,1,0,1,1,1,3,3,2,1,3,1,1,3,20,4,2,7.13,0


In [134]:
# # define X and y
# X = df5.drop('sgpa', axis=1)
# y = df5['sgpa']

In [135]:
df5.head()

,10th%,12thordiploma%,Internal,Prev cgpa,Gender,Location_urban,Pstatus,Mjob_civil,Mjob_Teacher/health,Fjob_civil,Fjob_Teacher/health,reason_reputation,reason_other,reason_closetohome,Famsup,Paid,Activities,HigherEdu,Internet,Medu,Fedu,traveltime,studytime,Famrel,Freetime,GoOut,Health,Age,Famsize,Failures,sgpa,Total_eamcet/ecet_grade
0,0.576271,0.745297,0.830508,0.864407,0,1,1,0,0,1,0,0,1,0,1,0,1,0,1,4,4,2,0,3,2,2,4,21,5,0,8.93,1
1,0.183116,0.669492,0.762712,0.677966,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,4,4,3,1,3,2,1,3,22,4,0,8.8,3
2,0.300066,0.440678,0.118644,0.245763,1,1,1,0,0,0,1,1,0,0,1,0,0,0,1,2,3,3,3,3,0,0,4,21,4,0,6.73,2
3,0.415254,0.567797,0.334373,0.720339,1,1,1,0,0,0,1,1,0,0,1,0,1,0,1,3,4,3,1,3,2,3,3,21,4,0,8.53,3
4,0.338983,0.082452,0.227964,0.220339,0,1,1,0,0,1,0,0,0,0,1,0,1,1,1,3,3,2,1,3,1,1,3,20,4,2,7.13,0


In [136]:
df5.corr()['sgpa'].sort_values() 

KeyError: 'sgpa'

In [ ]:
# define the SelectKBest transformer
kbest = SelectKBest(score_func=mutual_info_regression, k=5)

# select the best 5 columns using SelectKBest
X_best = kbest.fit_transform(df5, y)

# get the indices of the best 5 columns
best_indices = kbest.get_support(indices=True)

# get the column names of the best 5 columns
all_columns = df5.columns.tolist()
best_columns = [all_columns[i] for i in best_indices]

# subset the data using the best 5 columns
df5 = df5[best_columns]
df5

In [137]:
obj_cols = df5.select_dtypes(include=['object']).columns
df5[obj_cols] = df4[obj_cols].applymap(lambda x: x.strip() if isinstance(x, str) else x).astype(float)


In [138]:
df5.corr()['sgpa'].sort_values() 

Failures                  -0.433560
Gender                    -0.328436
Health                    -0.163104
Age                       -0.146809
Paid                      -0.126713
Famrel                    -0.121982
reason_closetohome        -0.111860
studytime                 -0.071572
Activities                -0.069841
Fjob_civil                -0.064992
Mjob_Teacher/health       -0.038664
Medu                      -0.019637
Internet                  -0.016035
traveltime                -0.015758
GoOut                     -0.006263
Mjob_civil                 0.012412
HigherEdu                  0.019110
10th%                      0.026939
Pstatus                    0.045164
Famsize                    0.059176
reason_reputation          0.085845
Fedu                       0.100474
Famsup                     0.109234
Fjob_Teacher/health        0.113142
Freetime                   0.132514
Location_urban             0.134602
reason_other               0.147282
Total_eamcet/ecet_grade    0

In [ ]:
# define the SelectKBest transformer
ctf7 = SelectKBest(score_func=mutual_info_regression, k=5)

# select the best 5 columns using SelectKBest
df5 = ctf7.fit_transform(df5, y)

df5 =pd.DataFrame(df5,columns=[ '12thordiploma%', 'Internal','Prev cgpa','sgpa','Total_eamcet/ecet_grade'])

df5

In [ ]:
ctf8=RandomForestRegressor()

## Create PipeLine

In [ ]:
pipe=Pipeline(
   [
       ('ctf1',ctf1),
       ('ctf2',ctf2),
       ('ctf3',ctf3),
       ('ctf4',ctf4),
       ('ctf5',ctf5),
       ('ctf6',ctf6),
       ('ctf7',ctf7),
       ('ctf8',ctf8)
   ])

In [ ]:
pipe.fit(X_train,y_train)

## Explore the PipeLine

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pipe.named_steps['ctf1']

In [ ]:
y_pred=pipe.predict(X_test) 

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("R-squared: {:.4f}".format(r2))
print("MSE score: {:.4f}".format(mse))
print("MAE score: {:.4f}".format(mae))
print("RMSE score: {:.4f}".format(rmse))


In [ ]:
# y_pred

## Cross Validation Score

In [ ]:
cross_val_score(pipe,X_train,y_train,cv=10,scoring='r2').mean()

## Pickling The Model File For Deployment

In [ ]:
import pickle
pickle.dump(pipe,open('sapp_final_algo.pkl','wb'))

In [ ]:
pipe.predict(pd.DataFrame([['319126510001','FEMALE',  21  ,  'Urban'   ,     4    ,    'T'   ,"higher education",'higher education','Homemaker','civil services: administrative or police', 'Other' , '30min - 1hr', '1- <2 hr' ,     0     , 'Yes'  ,  'No' ,    'Yes'    ,     'No'   ,    'Yes'  ,'excellent',   'high'  ,'3 - medium','5 - very good',  93.1   ,       96.3      ,   90.23   ,    9.4    ,    18238.0    ,      0     ,                        0]],
                    columns=['RollNo'      ,'Gender', 'Age', 'Location', 'Famsize', 'Pstatus',      'Medu'      ,       'Fedu'     ,   'Mjob'  ,                  'Fjob'                  , 'reason',  'traveltime', 'studytime', 'Failures','Famsup', 'Paid', 'Activities', 'HigherEdu', 'Internet', 'Famrel'  , 'Freetime',   'GoOut'  ,   'Health'    , '10th%' , '12thordiploma%', 'Internal','Prev cgpa',  'Eamcet Rank', 'Ecet Rank','Total_eamcet/ecet_grade']))